# Paso 9

* En la clase `Board`, crea el método `.isPossible()` que recibe la palabra a colocar, las coordenadas (`x`, `y`) y la dirección y devuelve la tupla (`True`, `message`) si es posible colocar dicha palabra sobre el tablero con la posición y dirección proporcionadas y la tupla (`False`, `message`) en caso contrario. La variable `message` es un string indicando el problema pertinente en caso de la primera entrada de la tupla ser `False` o indicando que la palabra puede situarse, en caso de ser `True`.
* Ten en cuenta las siguientes restricciones:
  - La primera palabra debe tener al menos una ficha situada en la casilla central.
  - La palabra no puede salirse de los límites del tablero.
  - Todas las palabras, a excepción de la primera, deben usar una ficha ya existente en el tablero.
  - No se puede situar una ficha en una casilla ya ocupada por otra ficha diferente.
  - Hay que colocar al menos una nueva ficha en el tablero
  - No puede haber una ficha al principio o al final de la palabra que se vaya a colocar sobre el tablero si ésta no pertenece a la palabra
* Para la creación del método anterior, te puede ser útil crear un método `.getLengthWord()` en la clase `Word` que te devuelva la longitud de la palabra. Para simplificar el código creado hasta el momento, sustituye cualquier línea de la forma `len(objWord.word)` por `objWord.getLengthWord()`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
class Pawns:
    
    def __init__(self):
        self.letters = []
    
    def addPawn(self, c):
        self.letters.append(c.upper())
    
    def addPawns(self, c, n):
        for _ in range(n):
            self.addPawn(c)
    
    def createBag(self):
        with open("/content/drive/MyDrive/Colab Notebooks/datasets/bag_of_pawns.csv") as f:
            count = 0
            for line in f:
                if count >= 1:
                    fi_tot = line.split(sep = ',')
                    self.addPawns(fi_tot[0], int(fi_tot[1]))
                count += 1

    def showPawns(self):
        f_pawns = self.getFrequency()
        f_pawns.showFrequency()

    def takeRandomPawn(self):
        from numpy import random
        return self.letters.pop(random.randint(0, self.getTotalPawns() -1 ))
    
    def getFrequency(self):
        f = FrequencyTable()
        for c in self.letters:
            f.update(c)
        return f
    
    def takePawn(self, c):
        self.letters.remove(c)
    
    def getTotalPawns(self):
        return len(self.letters)

class Word:

    def __init__(self, word = []):
        self.word = word

    def __str__(self):
        return "".join(self.word)
    
    def areEqual(self, w):
        return self.word == w.word
    
    def isEmpty(self):
        return self.getLengthWord() == 0
    
    @classmethod
    def readWord(cls):
        in_word = list(input().upper())
        return cls(in_word)
    
    @staticmethod
    def readWordFromFile(f):
        file_word = list(f.readline()[:-1])
        w = Word(file_word)
        return w

    def getFrequency(self):
        f = FrequencyTable()
        for c in self.word:
            f.update(c)
        return f

    def getLengthWord(self):
        return len(self.word)

class Dictionary:

    filepath = "/content/drive/MyDrive/Colab Notebooks/datasets/dictionary.txt"

    @staticmethod
    def validateWord(word):
        with open(Dictionary.filepath, 'r') as f:
            w = Word.readWordFromFile(f)
            while not word.areEqual(w) and not w.isEmpty():
                w = Word.readWordFromFile(f)
            if word.areEqual(w) and not w.isEmpty():
                return True
            else:
                return False
                
class FrequencyTable:

    def __init__(self):
        letters = []
        frequencies = []
        import csv
        with open("/content/drive/MyDrive/Colab Notebooks/datasets/bag_of_pawns.csv", "r") as f:
            reader = csv.reader(f)
            count = 0
            for row in reader:
                if count >= 1:
                    letters.append(row[0])
                count += 1
        self.letters = letters
        frequencies = [0] * len(letters)
        self.frequencies = frequencies
    
    def showFrequency(self):
        for i in range(len(self.frequencies)):
            if self.frequencies[i] != 0:
                print("{}: {}".format(self.letters[i], self.frequencies[i]))

    @staticmethod
    def isSubset(f1, f2):
        for i in range(len(f1.frequencies)):
            if f1.frequencies[i] > f2.frequencies[i]:
                return False
        return True
    
    def update(self, c):
        idx = self.letters.index(c)
        self.frequencies[idx] += 1

class Board:
    
    def __init__(self):
        board = [[" " for j in range(15)] for i in range(15)]
        self.board = board
        self.totalWords = 0
        self.totalPawns = 0
    
    def showBoard(self):
        count_row = 0
        
        for i in range(len(self.board) * 2 + 2):
            
            if i == 0:
                for j in range(len(self.board)):
                    print("  {}".format('0'+str(j) if j < 10 else str(j)), end = '')
                print()
            
            elif i % 2 == 0 :
                for j in range(len(self.board)):
                    if j == len(self.board) -1:
                        print("| {} | {}".format(self.board[count_row][j], 
                                                 '0'+str(count_row) if count_row < 10 else str(count_row) ),
                                                  end = "")
                    else:
                        print("| {} ".format(self.board[count_row][j]), end = "")
                count_row += 1
                print()
            
            else:
                print("+---"*15)
    
    def placeWord(self, player_pawns, word, x, y, dir):

        dir = dir.upper()

        for l in word.word:
            if l != self.board[x][y]:
                player_pawns.takePawn(l)
                self.totalPawns += 1
                self.board[x][y] = l

            if dir == 'V':
                x += 1
            if dir == 'H':
                y += 1
        self.totalWords += 1
    
    def isPossible(self, word, x, y, dir):
        
        message = ""
        x0 = x
        y0 = y
        # La primera palabra debe tener al menos una ficha situada en la casilla central
        if self.totalWords == 0:
            message = "Ninguna ficha para por la casilla central (7,7)"
            if dir == 'V':
                if y0 != 7:
                    return (False, message)
                elif x0 + word.getLengthWord() - 1 < 7 or x > 7:
                    return (False, message)

            if dir == 'H':
                if x0 != 7:
                    return (False, message)
                elif y0 + word.getLengthWord() - 1 < 7 or y > 7:
                    return (False, message)
        # La palabra no puede salirse de los límites del tablero          
        else:
            message = "La palabra se sale de los límites del tablero"
            if dir == 'V' and x0 + word.getLengthWord() - 1 >= 15:
                return (False, message)
            if dir == 'H' and y0 + word.getLengthWord() -1 >= 15:
                return (False, message)
            if x0 < 0 or x0 >= 15 or y0 < 0 or y >= 15:
                return (False, message)
            # Todas las palabras, a excepción de la primera, deben usar una ficha ya existente en el tablero.
            
            x = x0
            y = y0
            blanks = []
            for c in word.word:
                if self.board[x][y] == " ":
                    blanks.append(c)
                
                if dir == "V":
                    x += 1
                if dir == "H":
                    y += 1

            if len(blanks) == word.getLengthWord():
                message = "No se está utilizando ninguna ficha del tablero"
                return (False, message)

            # No se puede situar una ficha en una casilla ya ocupada por otra ficha diferente
            x = x0
            y = y0
            for c in word.word:
                if self.board[x][y] != " " and self.board[x][y] != c:
                    message = "Hay una ficha diferente ocupando una posición"
                    return (False, message)
                if dir == "V":
                    x += 1
                if dir == "H":
                    y += 1
                
            # Hay que colocar al menos una nueva ficha en el tablero
            x = x0
            y = y0
            matching = []
            for c in word.word:
                if self.board[x][y] == c:
                    matching.append(c)
                if dir == "V":
                    x += 1
                if dir == "H":
                    y += 1
            
            if len(matching) == word.getLengthWord():
                message = "No se está colocando ninguna ficha nueva en el tablero"
                return (False, message)
            
            # No puede haber una ficha al principio o al final de la palabra que se vaya a colocar sobre el tablero si ésta no pertenece a la palabra
            message = "Hay fichas adicionales a principio o final de palabra"
            x = x0
            y = y0
            if dir == "V" and ((x != 0 and self.board[x - 1][y] != " ") or (x + word.getLengthWord() != 14 and self.board[x + word.getLengthWord()][y] != " ")):
                return (False, message)
            if dir == "H" and ((y != 0 and self.board[x][y - 1] != " ") or (y + word.getLengthWord() != 14 and self.board[x][y + word.getLengthWord()] != " ")):
                return (False, message)

        message = "La palabra se puede situar en el tablero"
        return (True, message)

In [3]:
bag_of_pawns = Pawns()
player_pawns = Pawns()
bag_of_pawns.createBag()
for _ in range(7):
    player_pawns.addPawn(bag_of_pawns.takeRandomPawn())
player_pawns.showPawns()
board = Board()

D: 1
E: 1
I: 1
N: 2
S: 1
U: 1


In [5]:
x = int(input("x = "))
y = int(input("y = "))
new_word = Word.readWord()
dir = input()
print(board.isPossible(new_word, x, y, dir))
board.placeWord(player_pawns, new_word, x, y, dir)
board.showBoard()

player_pawns.showPawns()
print(player_pawns.getTotalPawns())
while(player_pawns.getTotalPawns() < 7):
  player_pawns.addPawn(bag_of_pawns.takeRandomPawn())

player_pawns.showPawns()
print(player_pawns.getTotalPawns())
print(board.totalPawns)
print(board.totalWords) 

x = 4
y = 3
DENP
V
(False, 'Hay una ficha diferente ocupando una posición')
  00  01  02  03  04  05  06  07  08  09  10  11  12  13  14
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   | 00
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   | 01
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   | 02
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   | 03
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---
|   |   |   | D |   |   |   |   |   |   |   |   |   |   |   | 04
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---
|   |   |   | E |   |   |   |   |   |   |   |   |   |   |   | 05
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---
|   |   |   | N |   |   |   |   |   |   |   | 